In [93]:
import pandas as pd
import numpy as np
import datetime as dt

In [111]:
checkindf = pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\CHECKINOUT.csv')
userdf = pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\USERINFO.csv')
eventlogdf= pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\acc_monitor_log.csv')
departmentdf = pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\DEPARTMENTS.csv')
machinesdf = pd.read_csv(r'C:\Users\NanaYawDarko\door_access_accra\data_exports\Machines.csv')


In [112]:
# get the columns of interest from the userdf
userdf_columns = ['USERID','name','lastname','email','DEFAULTDEPTID']
userdf = userdf[userdf_columns]
#Rename the columns to be more descriptive
userdf.rename(columns={'USERID':'userid','name':'firstname','email':'Email','DEFAULTDEPTID':'deptid'}, inplace=True)
userdf.describe()

,userid,deptid
count,313.000000,313.000000
mean,168.367412,48.974441
std,95.485311,8.009169
min,1.000000,1.000000
25%,87.000000,47.000000
50%,171.000000,47.000000
75%,251.000000,53.000000
max,329.000000,58.000000


In [113]:
# get the columns of interest from the checkindf

checkindf_columns = ['USERID','CHECKTIME','LOGID',]
checkindf = checkindf[checkindf_columns]
checkindf.rename(columns={'USERID':'userid','CHECKTIME':'logtime','LOGID':'logid'}, inplace=True)

In [114]:
# convert the logtime column to datetime format

checkindf['logtime'] = pd.to_datetime(checkindf['logtime'], format='%Y-%m-%d %H:%M:%S')
checkindf['logtime_year'] = checkindf['logtime'].dt.year

# filter the checkindf to only include records from 2024 to now
checkindf = checkindf[checkindf['logtime_year'] >= 2024]

checkindf.head()

,userid,logtime,logid,logtime_year
130,2,2024-03-08 11:26:59,1944,2024
131,2,2024-03-08 11:37:49,1955,2024
132,2,2024-03-08 11:56:10,1964,2024
133,2,2024-03-08 12:40:25,80011,2024
134,2,2024-03-08 12:48:26,143527,2024


In [115]:
# get the columns of interest from eventlogdf
eventlog_columns = ['id','time','device_name','state','event_type','event_point_name']
eventlogdf = eventlogdf[eventlog_columns]
eventlogdf.rename(columns={'id':'logid','time':'logtime'}, inplace=True)

In [116]:

eventlogdf['logtime'] = pd.to_datetime(eventlogdf['logtime'], format='%Y-%m-%d %H:%M:%S')
eventlogdf['logtime_year'] = eventlogdf['logtime'].dt.year
# filter the eventlogdf to only include records from 2024 to now
eventlogdf = eventlogdf[eventlogdf['logtime_year'] >= 2024]
eventlogdf.head()

,logid,logtime,device_name,state,event_type,event_point_name,logtime_year
76385,112113,2024-01-02 05:04:20,BLOCK B FIRST FLOOR,10,0,BLOCK B FIRST FLOOR-1,2024
76386,181610,2024-01-02 05:12:14,MAIN DOOR,10,0,MAIN DOOR-1,2024
76387,78044,2024-01-02 05:16:32,1ST FLOOR BLOCK A,10,0,1ST FLOOR BLOCK A-1,2024
76388,78045,2024-01-02 05:16:33,1ST FLOOR BLOCK A,10,0,1ST FLOOR BLOCK A-1,2024
76389,167219,2024-01-02 05:17:05,LAST FOOR,10,0,LAST FOOR-1,2024


In [117]:
# left join userdf to checkindf on userid
usercheckindf = userdf.merge(checkindf, on='userid', how='left')

# usercheckindf.groupby('name').size().reset_index(name='count').sort_values(by='count', ascending=False).head(10)
usercheckindf.sort_values(by='logtime', ascending=True).head(100).reset_index(drop=True)

,userid,firstname,lastname,Email,deptid,logtime,logid,logtime_year
0,123,Eric Atsu,Tenunya,NaN,52,2024-01-02 05:04:20,72468.0,2024.0
1,74,Juliana,Nkeasen,NaN,52,2024-01-02 05:12:14,140002.0,2024.0
2,74,Juliana,Nkeasen,NaN,52,2024-01-02 05:16:32,38399.0,2024.0
3,74,Juliana,Nkeasen,NaN,52,2024-01-02 05:16:33,38400.0,2024.0
4,74,Juliana,Nkeasen,NaN,52,2024-01-02 05:17:05,125749.0,2024.0
...,...,...,...,...,...,...,...,...
95,123,Eric Atsu,Tenunya,NaN,52,2024-01-02 07:58:47,72489.0,2024.0
96,18,Ewurama,Amponsah,NaN,47,2024-01-02 07:59:13,72490.0,2024.0
97,18,Ewurama,Amponsah,NaN,47,2024-01-02 08:00:10,97859.0,2024.0
98,60,Patrick,Tettey,patrick.tettey@amalitech.org,51,2024-01-02 08:01:59,38426.0,2024.0


In [134]:
# join checkinuserdf to eventlogdf on logid
door_access_df = usercheckindf.merge(eventlogdf, on=['logtime'], how='left', suffixes=('_checkin', '_eventlog'))

# display the first 10 rows of the merged dataframe
door_access_df.count()

userid                   232683
firstname                232683
lastname                 232683
Email                    175539
deptid                   232683
logtime                  232642
logid_checkin            232642
logtime_year_checkin     232642
logid_eventlog           232642
device_name              232642
state                    232642
event_type               232642
event_point_name         232642
logtime_year_eventlog    232642
dtype: int64

In [139]:
# door_access_df.head(10).reset_index(drop=True).sort_values(by=['logid_y'], ascending=True)
# convert the logid_checkin to int from float

door_access_df['logid_checkin'] = door_access_df['logid_checkin'].astype(str)
door_access_df['logid_checkin'] = door_access_df['logid_checkin'].astype(int, errors='ignore')
door_access_df.dtypes

userid                            int64
firstname                        object
lastname                         object
Email                            object
deptid                            int64
logtime                  datetime64[ns]
logid_checkin                    object
logtime_year_checkin            float64
logid_eventlog                  float64
device_name                      object
state                           float64
event_type                      float64
event_point_name                 object
logtime_year_eventlog           float64
dtype: object